In [ ]:
# main.py
"""
CW5 - MSCI151: MILP staffing models (Baseline, Scenario A, Scenario B, Fairness)
Author: (Your name)
Usage: python main.py
Notes:
 - This script implements the coursework models using PuLP + CBC (default).
 - Data are embedded (as per the coursework), but you may load them from Excel if you prefer.
 - Reference/CW5 PDF uploaded by user: /mnt/data/CW5_MSCI151_released.pdf
   (include that file in your submission folder / README). :contentReference[oaicite:1]{index=1}
"""

import pulp as pl
from collections import defaultdict
import os
import csv

# ----------------------------
# DATA (matches CW5 PDF)
# ----------------------------
baristas = ["Max", "Jiwa", "Fore", "Donna", "Paul"]
days = list(range(1, 7))      # 1..7 (Mon..Sun)
blocks = list(range(1, 5))    # 1..4 (each = 4-hour block)
H = 4                         # hours per block

cost = {"Max": 50000, "Jiwa": 50000, "Fore": 50000, "Donna": 150000, "Paul": 150000}
etype = {"Max": "P", "Jiwa": "P", "Fore": "P", "Donna": "F", "Paul": "F"}
minWeekly_default = {"Max": 12, "Jiwa": 12, "Fore": 12, "Donna": 36, "Paul": 36}

# availability a_{o,d} as per table in CW5
avail = {
    ("Max", 1): 8,  ("Max", 2): 8,  ("Max", 3): 8,  ("Max", 4): 0,  ("Max", 5): 8,  ("Max", 6): 4,  ("Max", 7): 4,
    ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0,
    ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0,
    ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8,
    ("Paul", 1): 12, ("Paul", 2): 8,  ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8,  ("Paul", 6): 12, ("Paul", 7): 12,
}

DAY_NAME = {1: "Mon", 2: "Tue", 3: "Wed", 4: "Thu", 5: "Fri", 6: "Sat", 7: "Sun"}
BLOCK_NAME = {1: "07:00–11:00", 2: "11:00–15:00", 3: "15:00–19:00", 4: "19:00–23:00"}

# reference to uploaded coursework pdf (local path)
CW5_PDF_PATH = "/mnt/data/CW5_MSCI151_released.pdf"


# ----------------------------
# Helper functions
# ----------------------------
def build_staffing_model(baristas, days, blocks, H, cost, avail, etype, minWeekly, allow_blocks=None, name="Staffing", sense=pl.LpMinimize):
    """
    Build baseline staffing model (flat keys y[(o,d,b)]).
    """
    model = pl.LpProblem(name, sense)

    if allow_blocks is None:
        allow_blocks = {d: set(blocks) for d in days}
    else:
        allow_blocks = {d: set(bs) for d, bs in allow_blocks.items()}

    # Decision variables
    y = pl.LpVariable.dicts(
        "y",
        [(o, d, b) for o in baristas for d in days for b in allow_blocks[d]],
        lowBound=0, upBound=1, cat="Binary"
    )

    # Objective: minimize total weekly cost
    model += pl.lpSum(H * cost[o] * y[(o, d, b)] for o in baristas for d in days for b in allow_blocks[d]), "TotalCost"

    # 1) Coverage: at least one barista in every allowed block/day
    for d in days:
        for b in allow_blocks[d]:
            model += pl.lpSum(y[(o, d, b)] for o in baristas) >= 1, f"cover_d{d}_b{b}"

    # 2) Per-day availability (hours) and contractual block caps
    for o in baristas:
        for d in days:
            # availability limit
            model += H * pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= avail[(o, d)], f"avail_{o}_{d}"
            # contractual daily block cap
            cap = 2 if etype[o] == "P" else 3
            model += pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= cap, f"blockCap_{o}_{d}"

    # 4) Weekly minimum hours
    for o in baristas:
        model += H * pl.lpSum(y[(o, d, b)] for d in days for b in allow_blocks[d]) >= minWeekly[o], f"weeklyMin_{o}"

    return model, y, allow_blocks


def extract_results(model, y, baristas, days, blocks, allow_blocks, H):
    """
    Extract solver status, objective, hours per barista, and schedule (daily x block).
    """
    status = pl.LpStatus[model.status]
    total_cost = pl.value(model.objective)

    hours_by_barista = {o: 0 for o in baristas}
    schedule = defaultdict(lambda: defaultdict(list))  # schedule[d][b] -> list of baristas

    for o in baristas:
        for d in days:
            for b in allow_blocks[d]:
                val = y[(o, d, b)].value()
                if val is not None and val > 0.5:
                    hours_by_barista[o] += H
                    schedule[d][b].append(o)

    return status, total_cost, hours_by_barista, schedule


def print_summary(title, status, total_cost, hours_by_barista, schedule):
    print("\n" + "=" * 72)
    print(title)
    print("=" * 72)
    print(f"Solver status: {status}")
    if total_cost is not None:
        print(f"Total weekly cost (IDR): {int(total_cost):,}")
    print("\nWeekly hours per barista:")
    for o in hours_by_barista:
        print(f"  - {o:6s}: {hours_by_barista[o]:2d} hours")

    print("\nDaily × Block schedule:")
    for d in sorted(schedule.keys()):
        print(f"  {DAY_NAME[d]}:")
        for b in sorted(schedule[d].keys()):
            who = ", ".join(schedule[d][b]) if schedule[d][b] else "(none)"
            print(f"    [{BLOCK_NAME[b]}] → {who}")
    print("=" * 72 + "\n")


def save_schedule_csv(filename, hours_by_barista, schedule, days, blocks):
    """
    Save summary CSV with two parts: hours per barista and daily schedule table.
    """
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["--- Weekly hours per barista ---"])
        writer.writerow(["Barista", "Hours"])
        for o, hrs in hours_by_barista.items():
            writer.writerow([o, hrs])
        writer.writerow([])
        writer.writerow(["--- Daily schedule ---"])
        header = ["Day/Block"] + [BLOCK_NAME[b] for b in blocks]
        writer.writerow(header)
        for d in days:
            row = [DAY_NAME[d]]
            for b in blocks:
                names = schedule.get(d, {}).get(b, [])
                row.append(";".join(names) if names else "")
            writer.writerow(row)


# ----------------------------
# Fairness model builder
# ----------------------------
def build_fairness_model(baristas, days, blocks, H, cost, avail, etype, minWeekly, baseline_cost, allow_blocks=None, name="Fairness"):
    """
    Build fairness MILP:
    - minimize Hmax - Hmin
    - subject to coverage, availability, contractual caps, weekly minima
    - plus budget cap: total_cost <= 1.02 * baseline_cost
    Uses flat y[(o,d,b)] variables.
    """
    model = pl.LpProblem(name, pl.LpMinimize)

    if allow_blocks is None:
        allow_blocks = {d: set(blocks) for d in days}
    else:
        allow_blocks = {d: set(bs) for d, bs in allow_blocks.items()}

    # Binary assignment vars
    y = pl.LpVariable.dicts("y", [(o, d, b) for o in baristas for d in days for b in allow_blocks[d]], lowBound=0, upBound=1, cat="Binary")

    # Continuous hours vars and extrema
    hours_o = pl.LpVariable.dicts("hours", baristas, lowBound=0, cat="Continuous")
    Hmax = pl.LpVariable("Hmax", lowBound=0, cat="Continuous")
    Hmin = pl.LpVariable("Hmin", lowBound=0, cat="Continuous")

    # Objective: minimize gap
    model += Hmax - Hmin, "FairnessGap"

    # Coverage
    for d in days:
        for b in allow_blocks[d]:
            model += pl.lpSum(y[(o, d, b)] for o in baristas) >= 1, f"cover_d{d}_b{b}"

    # Daily availability and block caps
    for o in baristas:
        for d in days:
            model += H * pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= avail[(o, d)], f"avail_{o}_{d}"
            cap = 2 if etype[o] == "P" else 3
            model += pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= cap, f"blockCap_{o}_{d}"

    # Weekly minima
    for o in baristas:
        model += H * pl.lpSum(y[(o, d, b)] for d in days for b in allow_blocks[d]) >= minWeekly[o], f"weeklyMin_{o}"

    # Link hours_o and bound to Hmax/Hmin
    for o in baristas:
        model += hours_o[o] == H * pl.lpSum(y[(o, d, b)] for d in days for b in allow_blocks[d]), f"hours_link_{o}"
        model += hours_o[o] <= Hmax, f"leq_Hmax_{o}"
        model += hours_o[o] >= Hmin, f"geq_Hmin_{o}"

    # Budget cap (≤ 1.02 * baseline_cost)
    total_cost = pl.lpSum(H * cost[o] * y[(o, d, b)] for o in baristas for d in days for b in allow_blocks[d])
    model += total_cost <= 1.02 * baseline_cost, "budget_cap"

    return model, y, allow_blocks


# ----------------------------
# Scenario runners
# ----------------------------
def solve_baseline():
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, name="Baseline"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)  # 5 min
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_close_at_19():
    allow_blocks = {d: {1, 2, 3} for d in days}
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, allow_blocks=allow_blocks, name="CloseAt19"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_pt_min16():
    minWeekly_pt16 = {o: (16 if etype[o] == "P" else 36) for o in baristas}
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_pt16, name="PTMin16"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_fairness(baseline_cost):
    model, y, allow_blocks = build_fairness_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, baseline_cost, name="Fairness"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)



# ----------------------------
# Main execution
# ----------------------------
def main():
    print("CW5 MILP models using PuLP (CBC).")

    # Baseline
    baseline_model, (status_base, base_cost, base_hours, base_sched) = solve_baseline()
    print_summary("BASELINE (open 07:00–23:00)", status_base, base_cost, base_hours, base_sched)
    save_schedule_csv("results/baseline_schedule.csv", base_hours, base_sched, days, blocks)

    # Scenario A: close at 19:00 (remove block 4)
    sA_model, (status_sA, sA_cost, sA_hours, sA_sched) = solve_close_at_19()
    print_summary("SCENARIO A — CLOSE AT 19:00 (remove block 4)", status_sA, sA_cost, sA_hours, sA_sched)
    save_schedule_csv("results/close19_schedule.csv", sA_hours, sA_sched, days, [1, 2, 3])  # only 3 blocks

    # Scenario B: part-time weekly minimum = 16 hours
    sB_model, (status_sB, sB_cost, sB_hours, sB_sched) = solve_pt_min16()
    print_summary("SCENARIO B — PART-TIME MIN = 16 HOURS", status_sB, sB_cost, sB_hours, sB_sched)
    save_schedule_csv("results/ptmin16_schedule.csv", sB_hours, sB_sched, days, blocks)

    # Fairness model (≤ 102% of baseline cost)
    fairness_model, (status_fair, fair_cost, fair_hours, fair_sched) = solve_fairness(base_cost)
    print_summary("FAIRNESS VARIANT (minimize Hmax-Hmin; cost ≤ 102% of baseline)", status_fair, fair_cost, fair_hours, fair_sched)
    save_schedule_csv("results/fairness_schedule.csv", fair_hours, fair_sched, days, blocks)

    # Quick cost summary printed
    print("--- COST SUMMARY (IDR) ---")
    print(f"Baseline:  {int(base_cost):,} (status: {status_base})")
    print(f"Close@19:  {int(sA_cost):,} (status: {status_sA})")
    print(f"PT min16:  {int(sB_cost):,} (status: {status_sB})")
    print(f"Fairness:  {int(fair_cost):,} (status: {status_fair})")
    print("--------------------------\n")

    # remind user of CW5 pdf path for report
    print("Coursework PDF (for your report / README):", CW5_PDF_PATH)


if __name__ == "__main__":
    main()
